<a href="https://colab.research.google.com/github/keymemory/Machine_Learning_Second/blob/main/ML2_day4_AutoEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[AutoEncoder를 이용한 MNIST Reconstruction]

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# MNIST 데이터 다운로드
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

ModuleNotFoundError: ignored

In [ ]:
# 학습에 필요한 설정값들 정의
learning_rate = 0.02
training_epochs = 50    # 반복횟수
batch_size = 256        # 배치개수
display_step = 1        # 손실함수 출력 주기
examples_to_show = 10   # 보여줄 MNIST Reconstruction 이미지 개수
input_size = 784        # 28*28
hidden1_size = 256 
hidden2_size = 128

In [ ]:
# 입력을 받기 위한 플레이스홀더 정의
# Autoencoder는 Unsupervised Learning이므로 타겟 레이블(label) y가 필요하지 않다.
x = tf.placeholder(tf.float32, shape=[None, input_size])

In [ ]:
# Autoencoder 구조 정의
def build_autoencoder(x):
  # 인코딩(Encoding) - 784 -> 256 -> 128
  W1 = tf.Variable(tf.random_normal(shape=[input_size, hidden1_size]))
  b1 = tf.Variable(tf.random_normal(shape=[hidden1_size]))
  H1_output = tf.nn.sigmoid(tf.matmul(x,W1) + b1)
  W2 = tf.Variable(tf.random_normal(shape=[hidden1_size, hidden2_size]))
  b2 = tf.Variable(tf.random_normal(shape=[hidden2_size]))
  H2_output = tf.nn.sigmoid(tf.matmul(H1_output,W2) + b2)
  # 디코딩(Decoding) 128 -> 256 -> 784
  W3 = tf.Variable(tf.random_normal(shape=[hidden2_size, hidden1_size]))
  b3 = tf.Variable(tf.random_normal(shape=[hidden1_size]))
  H3_output = tf.nn.sigmoid(tf.matmul(H2_output,W3) + b3)
  W4 = tf.Variable(tf.random_normal(shape=[hidden1_size, input_size]))
  b4 = tf.Variable(tf.random_normal(shape=[input_size]))
  reconstructed_x = tf.nn.sigmoid(tf.matmul(H3_output,W4) + b4)

  return reconstructed_x

In [ ]:
# Autoencoder 선언
y_pred = build_autoencoder(x)
# 타겟 데이터는 인풋데이터와 같다.
y_true = x

In [ ]:
# 손실함수와 옵티마이저 정의
loss = tf.reduce_mean(tf.pow(y_true - y_pred, 2)) # MSE(Mean of Squared Error) 손실함수
train_step = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

In [ ]:
# 세션을 열고 그래프 실행
with tf.Session() as sess:
  # 변수들의 초기값 할당
  sess.run(tf.global_variables_initializer())

  # 지정된 횟수만큼 최적화 수행
  for epoch in range(training_epochs):
    # 전체 배치 호출
    total_batch = int(mnist.train.num_examples/batch_size)
    # 모든 배치들에 대해서 최적화 수행
    for i in range(total_batch):
      batch_xs, batch_ys = mnist.train.next_batch(batch_size)
      _, current_loss = sess.run([train_step, loss], feed_dict={x: batch_xs})
    # 지정된 epoch마다 학습 결과 출력
    if epoch % display_step == 0:
      print("반복(Epoch): %d, 손실 함수(Loss): %f" % ((epoch+1), current_loss))

  # 테스트 데이터로 Reconstruction 수행
  reconstructed_result = sess.run(y_pred, feed_dict={x: 
                   mnist.test.images[:examples_to_show]})
  # 원본 MNIST 데이터와 Reconstruction 결과 비교
  f, a = plt.subplots(2, 10, figsize=(10, 2))
  for i in range(examples_to_show):
    a[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
    a[1][i].imshow(np.reshape(reconstructed_result[i], (28, 28)))
  f.savefig('reconstructed_mnist_image.png')  # reconstruction 결과 png로 저장
  f.show()
  plt.draw()
  plt.waitforbuttonpress()

NameError: ignored